In [2]:
import pandas as pd 
import numpy as np
import os
from alpaca_trade_api import REST
import pygsheets


client = pygsheets.authorize(service_file='algoquant-351309-ae021cd7392e.json')

sh = client.open_by_key('1hPxsuwdDvpZpQYG2yi_djwcDG1RDAw1RcS-QZ5prUPo')

wk1 = sh[0] # open first worksheet of spreadsheet or sh.sheet1


os.environ['APCA_API_KEY_ID'] = 'PKCHDD6J9DQDE7BZZ1UV'
os.environ['APCA_API_SECRET_KEY'] = 'lQ5hWAXPrGx2F0jEO0eiDULu1w2cOiG2liCeX101'
os.environ['APCA_API_BASE_URL'] = 'https://paper-api.alpaca.markets'
api = REST()

from alpaca_trade_api.rest import REST, TimeFrame
api = REST()

result = api.get_activities()

result_df = pd.DataFrame()

print(result[0]) # cum_qty, price, symbol, transaction_time

result_df = pd.DataFrame(columns = ['Qty','price','symbol','transaction_time'])

qty_list = []
price_list = []
symbol_list = []
transaction_time_list = []
type_list = []

for res in result:
	qty_list.append(res.cum_qty)
	price_list.append(res.price)
	symbol_list.append(res.symbol)
	transaction_time_list.append(res.transaction_time)
	type_list.append(res.side)



result_df = pd.DataFrame({'Qty': qty_list, 'Price': price_list, 'Symbol': symbol_list, 'Transaction_time': transaction_time_list, 'Type': type_list})
print(result_df)

df_buy = result_df[result_df.Type == 'buy']

df_sell = result_df[result_df.Type == 'sell']

df = pd.merge(df_buy, df_sell, on = 'Symbol', how = 'right', suffixes = ['_buy','_sell'])

print(df_sell)

AccountActivity({   'activity_type': 'FILL',
    'cum_qty': '1',
    'id': '20220511151628481::b30c3edb-093a-4c92-9f14-1fa981e7d9a0',
    'leaves_qty': '0',
    'order_id': '4273eb5d-062f-454d-a2e2-4494993d0137',
    'order_status': 'filled',
    'price': '35.2',
    'qty': '1',
    'side': 'buy',
    'symbol': 'IAU',
    'transaction_time': '2022-05-11T19:16:28.481084Z',
    'type': 'fill'})
   Qty   Price Symbol                 Transaction_time  Type
0    1    35.2    IAU 2022-05-11 19:16:28.481084+00:00   buy
1    1   35.11    IAU 2022-05-11 14:16:11.761519+00:00   buy
2    3   79.75    XLE 2022-05-09 14:15:25.778872+00:00  sell
3    1   70.77    XLU 2022-05-03 19:39:27.207603+00:00   buy
4    1   35.47    IAU 2022-05-03 19:39:26.978201+00:00   buy
..  ..     ...    ...                              ...   ...
65   1  109.76    VYM 2022-02-22 15:54:11.522403+00:00   buy
66   1   52.28   DGRO 2022-02-22 15:54:11.429111+00:00   buy
67   1   40.59   SPYV 2022-02-22 15:54:11.385972+00:00 

In [11]:
testing = df_sell.sort_values(by = 'Transaction_time')
testing

,Qty,Price,Symbol,Transaction_time,Type
60,1,118.64,DVY,2022-02-24 15:07:34.451780+00:00,sell
52,1,145.74,VTV,2022-03-17 14:28:18.252160+00:00,sell
49,1,183.01,GLD,2022-04-12 14:00:13.580350+00:00,sell
48,1,37.27,IAU,2022-04-12 14:00:13.628443+00:00,sell
44,1,80.99,XLE,2022-04-19 14:27:18.021648+00:00,sell
42,2,80.14,XLP,2022-04-22 14:28:13.882265+00:00,sell
2,3,79.75,XLE,2022-05-09 14:15:25.778872+00:00,sell


In [75]:
testing['Price'] = testing.Price.astype('float')
df_buy['Price'] = df_buy.Price.astype('float')

/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3849458565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buy['Price'] = df_buy.Price.astype('float')


Qty_buy	Price_buy	Symbol	Transaction_time_buy	Type_buy	Qty_sell	Price_sell	Transaction_time_sell	Type_sell	Profit	Holding_Period	

In [111]:
output_frame = pd.DataFrame(columns = ['Symbol', 'Qty','Avg_cost','Avg_holding_period',
                             'Earliest_buy_time','Latest_buy_time','Sell_time','Profit_per_unit','Total Profit'])

for sym in testing.Symbol.unique():
    print(sym)
    buy = df_buy.loc[df_buy.Symbol == sym]
    sell = testing.loc[testing.Symbol == sym]

    obs = [] # completed sell\'s index
    for i, row in sell.iterrows():
        output_dic = {}
        if i not in obs:
            out = buy.loc[(buy.Transaction_time < row.Transaction_time)]
            idx = [j for j in out.index if j not in obs]
            out = out.loc[idx]
            assert out.shape[0] == int(row.Qty)
            
            # Avg_cost
            print(row.Price - out.groupby('Symbol').Price.mean())
            print(row.Transaction_time - out.groupby('Symbol').Transaction_time.mean())
            output_dict = {'Symbol': sym, 
                           'Qty': int(row.Qty), 
                           'Avg_cost': round(out.groupby('Symbol').Price.mean()[0],2),
                           'Avg_holding_period': (row.Transaction_time - out.groupby('Symbol').Transaction_time.mean())[0],
                           'Earliest_buy_time': out.groupby('Symbol').Transaction_time.min()[0],
                           'Latest_buy_time': out.groupby('Symbol').Transaction_time.max()[0],
                           'Sell_time': row.Transaction_time,
                           'Profit_per_unit': round(row.Price - out.groupby('Symbol').Price.mean()[0],2),
                           'Total Profit': round((row.Price - out.groupby('Symbol').Price.mean())[0] * int(row.Qty),2)}
            zz = zz.append(output_dict, ignore_index = True)
            
            if len(idx) > 1:
                for ix in idx:
                    obs.append(ix)
            else:
                obs.append(idx[0])
        

DVY
Symbol
DVY   -3.85
Name: Price, dtype: float64
Symbol
DVY   0 days 23:58:03.187266976
Name: Transaction_time, dtype: timedelta64[ns]
VTV
Symbol
VTV    2.48
Name: Price, dtype: float64
Symbol
VTV   22 days 22:34:07.105072896
Name: Transaction_time, dtype: timedelta64[ns]
GLD
Symbol
GLD    4.88
Name: Price, dtype: float64
Symbol
GLD   26 days 23:40:30.702008880
Name: Transaction_time, dtype: timedelta64[ns]
IAU
Symbol
IAU    0.99
Name: Price, dtype: float64
Symbol
IAU   26 days 23:40:30.800062072
Name: Transaction_time, dtype: timedelta64[ns]
XLE
Symbol
XLE    13.17
Name: Price, dtype: float64
Symbol
XLE   54 days 23:17:46.882015872
Name: Transaction_time, dtype: timedelta64[ns]


/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zz = zz.append(output_dict, ignore_index = True)
/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zz = zz.append(output_dict, ignore_index = True)
/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zz = zz.append(output_dict, ignore_index = True)
/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

Symbol
XLE    6.023333
Name: Price, dtype: float64
Symbol
XLE   13 days 00:10:38.754705344
Name: Transaction_time, dtype: timedelta64[ns]
XLP
Symbol
XLP    5.64
Name: Price, dtype: float64
Symbol
XLP   56 days 10:56:12.513088936
Name: Transaction_time, dtype: timedelta64[ns]


/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zz = zz.append(output_dict, ignore_index = True)
/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zz = zz.append(output_dict, ignore_index = True)
/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_16961/3872642647.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zz = zz.append(output_dict, ignore_index = True)


In [112]:
zz

,Symbol,Qty,Avg_cost,Avg_holding_period,Earliest_buy_time,Latest_buy_time,Sell_time,Profit_per_unit,Total Profit
0,DVY,1,122.49,0 days 23:58:03.187266976,2022-02-23 15:09:31.264513+00:00,2022-02-23 15:09:31.264513+00:00,2022-02-24 15:07:34.451780+00:00,-3.85,-3.85
1,VTV,1,143.26,22 days 22:34:07.105072896,2022-02-22 15:54:11.147087+00:00,2022-02-22 15:54:11.147087+00:00,2022-03-17 14:28:18.252160+00:00,2.48,2.48
2,GLD,1,178.13,26 days 23:40:30.702008880,2022-03-16 14:19:42.878341+00:00,2022-03-16 14:19:42.878341+00:00,2022-04-12 14:00:13.580350+00:00,4.88,4.88
3,IAU,1,36.28,26 days 23:40:30.800062072,2022-03-16 14:19:42.828381+00:00,2022-03-16 14:19:42.828381+00:00,2022-04-12 14:00:13.628443+00:00,0.99,0.99
4,XLE,1,67.82,54 days 23:17:46.882015872,2022-02-23 15:09:31.139632+00:00,2022-02-23 15:09:31.139632+00:00,2022-04-19 14:27:18.021648+00:00,13.17,13.17
5,XLE,3,73.73,13 days 00:10:38.754705344,2022-04-25 14:01:45.697232+00:00,2022-04-27 14:12:19.419722+00:00,2022-05-09 14:15:25.778872+00:00,6.02,18.07
6,XLP,2,74.5,56 days 10:56:12.513088936,2022-02-24 15:00:26.692983+00:00,2022-02-25 16:03:36.045369+00:00,2022-04-22 14:28:13.882265+00:00,5.64,11.28


In [113]:
assert 1 == 1

In [26]:
[i for i in df_sell.index if i not in [1,2]]

[42, 44, 48, 49, 52, 60]

In [49]:
df_buy.tail(10)

,Qty,Price,Symbol,Transaction_time,Type
59,1,105.63,VYM,2022-02-24 15:08:14.527113+00:00,buy
61,1,73.65,XLP,2022-02-24 15:00:26.692983+00:00,buy
62,1,122.49,DVY,2022-02-23 15:09:31.264513+00:00,buy
63,1,67.82,XLE,2022-02-23 15:09:31.139632+00:00,buy
64,1,151.4,IVE,2022-02-22 15:54:11.536056+00:00,buy
65,1,109.76,VYM,2022-02-22 15:54:11.522403+00:00,buy
66,1,52.28,DGRO,2022-02-22 15:54:11.429111+00:00,buy
67,1,40.59,SPYV,2022-02-22 15:54:11.385972+00:00,buy
68,1,39.19,XLF,2022-02-22 15:54:11.161365+00:00,buy
69,1,143.26,VTV,2022-02-22 15:54:11.147087+00:00,buy


In [48]:
df_sell.tail()

,Qty,Price,Symbol,Transaction_time,Type
44,1,80.99,XLE,2022-04-19 14:27:18.021648+00:00,sell
48,1,37.27,IAU,2022-04-12 14:00:13.628443+00:00,sell
49,1,183.01,GLD,2022-04-12 14:00:13.580350+00:00,sell
52,1,145.74,VTV,2022-03-17 14:28:18.252160+00:00,sell
60,1,118.64,DVY,2022-02-24 15:07:34.451780+00:00,sell


In [50]:
df_buy.loc[df_buy.Transaction_time < df_sell.Transaction_time.max()]

,Qty,Price,Symbol,Transaction_time,Type
3,1,70.77,XLU,2022-05-03 19:39:27.207603+00:00,buy
4,1,35.47,IAU,2022-05-03 19:39:26.978201+00:00,buy
5,1,174.11,GLD,2022-05-03 19:39:26.964169+00:00,buy
6,1,70.76,XLU,2022-05-03 14:00:23.418826+00:00,buy
7,1,76.34,XLP,2022-05-03 14:00:23.404635+00:00,buy
...,...,...,...,...,...
65,1,109.76,VYM,2022-02-22 15:54:11.522403+00:00,buy
66,1,52.28,DGRO,2022-02-22 15:54:11.429111+00:00,buy
67,1,40.59,SPYV,2022-02-22 15:54:11.385972+00:00,buy
68,1,39.19,XLF,2022-02-22 15:54:11.161365+00:00,buy


# Unrealized Profits

In [1]:
import pandas as pd 
import numpy as np
import os
from alpaca_trade_api import REST
import pygsheets
from alpaca_trade_api.rest import REST, TimeFrame

os.environ['APCA_API_KEY_ID'] = 'PKCHDD6J9DQDE7BZZ1UV'
os.environ['APCA_API_SECRET_KEY'] = 'lQ5hWAXPrGx2F0jEO0eiDULu1w2cOiG2liCeX101'
os.environ['APCA_API_BASE_URL'] = 'https://paper-api.alpaca.markets'

api = REST()

In [2]:
api.get_portfolio_history().equity

[9997.19,
 10050.32,
 9954.49,
 9934.18,
 9969.7,
 10078,
 9992.55,
 9998.91,
 9898.385,
 9861.79,
 9862.04,
 9834.84,
 9875.18,
 9899.77,
 9942.43,
 9848.4,
 9861.94,
 9870.39,
 9921.32,
 9954.93,
 9965.64,
 9994.5,
 10050.27]

In [3]:
api.get_account()

Account({   'account_blocked': False,
    'account_number': 'PA3L5EBVA6LQ',
    'accrued_fees': '0',
    'buying_power': '15219.76',
    'cash': '5169.49',
    'created_at': '2022-02-22T15:40:56.226622Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '10050.27',
    'id': '089ead34-9c24-4a08-99fb-82a83d2403c8',
    'initial_margin': '2440.39',
    'last_equity': '10050.27',
    'last_maintenance_margin': '1464.23',
    'long_market_value': '4880.78',
    'maintenance_margin': '1464.234',
    'multiplier': '2',
    'non_marginable_buying_power': '5169.49',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '10050.27',
    'regt_buying_power': '15219.76',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '9994.5',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [4]:
acc_start = api.get_account().created_at.strftime('%Y-%m-%d')
acc_start

'2022-02-22'

In [5]:
def get_trading_cal(api):
    cal = api.get_calendar()
    cal_ls = []
    for c in cal:
        cal_ls.append(c.date)
      
    cal_ls = pd.Series(cal_ls)
    return cal_ls

cal_ls = get_trading_cal(api)
cal_ls

0       1970-01-02
1       1970-01-05
2       1970-01-06
3       1970-01-07
4       1970-01-08
           ...    
15131   2029-12-18
15132   2029-12-19
15133   2029-12-20
15134   2029-12-21
15135   2029-12-24
Length: 15136, dtype: datetime64[ns]

In [6]:
trading_days = cal_ls[(cal_ls >= acc_start) & (cal_ls < dt.datetime.strftime(dt.datetime.today(), '%Y-%m-%d'))]
trading_days = trading_days.apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))

NameError: name 'dt' is not defined

In [7]:
for trade in trading_days:
    result = api.get_portfolio_history(trade)
    profit_loss = result.profit_loss
    pnl_pct = result.profit_loss_pct
    timestamp = len([dt.datetime.fromtimestamp(val).strftime('%Y-%m-%d %H:%M') for val in result.timestamp])
    if timestamp >= 50:
        print(trade, timestamp)

NameError: name 'trading_days' is not defined

In [244]:
df_trade = pd.DataFrame(columns = ['timestamp','equity','pnl','pnl (pct)'])
for trade in trading_days:
    result = api.get_portfolio_history(trade)
    profit_loss = result.profit_loss
    pnl_pct = result.profit_loss_pct
    timestamp = [dt.datetime.fromtimestamp(val).strftime('%Y-%m-%d %H:%M') for val in result.timestamp]
    df_trade = pd.concat([df_trade, pd.DataFrame({'timestamp': timestamp, 'equity': result.equity, 'pnl':profit_loss, 'pnl (pct)': pnl_pct })], axis = 0)

In [245]:
df_trade = df_trade.sort_values(by = 'timestamp').drop_duplicates(subset = ['timestamp'],ignore_index=True, keep = 'first')

In [246]:
df_trade

,timestamp,equity,pnl,pnl (pct)
0,2022-02-23 09:00,9997.11,-2.89,0.0
1,2022-02-24 09:00,9988.83,-8.28,-0.000828
2,2022-02-25 09:00,9988.71,-0.1,-0.00001
3,2022-02-26 09:00,10020.54,31.83,0.003177
4,2022-03-01 09:00,10016.27,-4.27,0.002759
...,...,...,...,...
2394,2022-05-28 03:56,NaN,NaN,NaN
2395,2022-05-28 03:57,NaN,NaN,NaN
2396,2022-05-28 03:58,NaN,NaN,NaN
2397,2022-05-28 03:59,NaN,NaN,NaN


In [247]:
df_trade['date'] = df_trade['timestamp'].apply(lambda x: x[:10])
df_agg = df_trade.groupby('date').last()
df_agg = df_agg.reset_index().drop('timestamp', axis = 1)
df_agg

,date,equity,pnl,pnl (pct)
0,2022-02-23,9997.11,-2.89,0.000000
1,2022-02-24,9988.83,-8.28,-0.000828
2,2022-02-25,9988.71,-0.10,-0.000010
3,2022-02-26,10020.54,31.83,0.003177
4,2022-03-01,10016.27,-4.27,0.002759
...,...,...,...,...
66,2022-05-24,9920.69,-0.63,-0.000063
67,2022-05-25,9913.14,-8.18,-0.000824
68,2022-05-26,9896.09,25.70,0.002604
69,2022-05-27,9890.36,28.32,0.002872


In [248]:
open_positions[0]

Position({   'asset_class': 'us_equity',
    'asset_id': 'd058cc01-cdc1-4234-b3c3-e0f975677344',
    'asset_marginable': True,
    'avg_entry_price': '20.86',
    'change_today': '0',
    'cost_basis': '20.86',
    'current_price': '21.68',
    'exchange': 'ARCA',
    'lastday_price': '21.68',
    'market_value': '21.68',
    'qty': '1',
    'qty_available': '1',
    'side': 'long',
    'symbol': 'AMJ',
    'unrealized_intraday_pl': '0',
    'unrealized_intraday_plpc': '0',
    'unrealized_pl': '0.82',
    'unrealized_plpc': '0.0393096836049856'})

In [249]:
open_positions = api.list_positions()

asset_class_ls = []
avg_entry_price_ls = []
change_today_ls = []
cost_basis_ls = []
current_price_ls = []
exchange_ls = []
lastday_price_ls = []
market_value_ls = []
qty_ls = []
side_ls = []
symbol_ls = []
unrealized_intraday_pl_ls = []
unrealized_intraday_plpc_ls = []
unrealized_pl_ls = []
unrealized_plpc_ls = []

for pos in open_positions:
    asset_class_ls.append(pos.asset_class)
    avg_entry_price_ls.append(pos.avg_entry_price)
    change_today_ls.append(pos.change_today)
    cost_basis_ls.append(pos.cost_basis)
    current_price_ls.append(pos.current_price)
    exchange_ls.append(pos.exchange)
    lastday_price_ls.append(pos.lastday_price)
    market_value_ls.append(pos.market_value)
    qty_ls.append(pos.qty)
    side_ls.append(pos.side)
    symbol_ls.append(pos.symbol)
    unrealized_intraday_pl_ls.append(pos.unrealized_intraday_pl)
    unrealized_intraday_plpc_ls.append(pos.unrealized_intraday_plpc)
    unrealized_pl_ls.append(pos.unrealized_pl)
    unrealized_plpc_ls.append(pos.unrealized_plpc)
    
op_df = pd.DataFrame({'Symbol': symbol_ls, 
                      'Asset_class': asset_class_ls, 
                      'Qty': qty_ls,
                      'Avg_entry_price': avg_entry_price_ls,
                      'Change_today': change_today_ls,
                      'Cost_basis': cost_basis_ls, 
                      'Current_price': current_price_ls,
                      'Exchange': exchange_ls,
                      'Lastday_price': lastday_price_ls,
                      'Market_value': market_value_ls,
                      'Side': side_ls,
                      'Unrealized_intraday_pl ($)': unrealized_intraday_pl_ls,
                      'Unrealized_intraday_pl (%)': unrealized_intraday_plpc_ls,
                      'Unrealized_pl ($)': unrealized_pl_ls,
                      'Unrealized_pl (%)': unrealized_plpc_ls
                     })

op_df['Avg_entry_price'] = op_df['Avg_entry_price'].astype('float')
op_df['Change_today'] = op_df['Change_today'].astype('float')
op_df['Current_price'] = op_df['Current_price'].astype('float')
op_df['Lastday_price'] = op_df['Lastday_price'].astype('float')
op_df['Market_value'] = op_df['Market_value'].astype('float')
op_df['Unrealized_intraday_pl ($)'] = op_df['Unrealized_intraday_pl ($)'].astype('float')
op_df['Unrealized_intraday_pl (%)'] = op_df['Unrealized_intraday_pl (%)'].astype('float')
op_df['Unrealized_pl ($)'] = op_df['Unrealized_pl ($)'].astype('float')
op_df['Unrealized_pl (%)'] = op_df['Unrealized_pl (%)'].astype('float')
op_df['Qty'] = op_df['Qty'].astype('int')

op_df['Avg_entry_price'] = op_df.Avg_entry_price.round(2)

op_df = op_df.sort_values(by = 'Unrealized_pl (%)', ascending = False)
op_df['Unrealized_pl (%)'] = op_df['Unrealized_pl (%)'].mul(100)
op_df['Unrealized_intraday_pl (%)'] = op_df['Unrealized_intraday_pl (%)'].mul(100)

op_df

,Symbol,Asset_class,Qty,Avg_entry_price,Change_today,Cost_basis,Current_price,Exchange,Lastday_price,Market_value,Side,Unrealized_intraday_pl ($),Unrealized_intraday_pl (%),Unrealized_pl ($),Unrealized_pl (%)
18,DVY,us_equity,5,122.34,0.006890,611.68,130.06,NASDAQ,129.17,650.30,long,4.45,0.689014,38.62,6.313759
2,AMJ,us_equity,1,20.86,0.005535,20.86,21.80,ARCA,21.68,21.80,long,0.12,0.553506,0.94,4.506232
14,XLU,us_equity,8,72.06,0.007007,576.46,74.73,ARCA,74.21,597.84,long,4.16,0.700714,21.38,3.708844
5,VYM,us_equity,2,107.70,0.006422,215.39,111.26,ARCA,110.55,222.52,long,1.42,0.642243,7.13,3.310274
21,SCHD,us_equity,1,76.58,0.008215,76.58,78.55,ARCA,77.91,78.55,long,0.64,0.821461,1.97,2.572473
8,VTV,us_equity,1,142.90,0.007554,142.9,144.05,ARCA,142.97,144.05,long,1.08,0.755403,1.15,0.804759
9,VOE,us_equity,1,144.67,0.011059,144.67,145.36,ARCA,143.77,145.36,long,1.59,1.105933,0.69,0.476948
10,SDY,us_equity,2,126.72,0.007057,253.44,127.01,ARCA,126.12,254.02,long,1.78,0.705677,0.58,0.228851
17,SPYV,us_equity,1,40.59,0.008748,40.59,40.36,ARCA,40.01,40.36,long,0.35,0.874781,-0.23,-0.566642
11,IVE,us_equity,1,151.40,0.009324,151.4,150.46,ARCA,149.07,150.46,long,1.39,0.932448,-0.94,-0.620872


In [250]:
op_df.shape[0] % 5

3

In [251]:
np.arange(0,op_df.shape[0], 5)

array([ 0,  5, 10, 15, 20])

In [27]:
# get_data
import datetime as dt

def get_trading_cal(api):
    cal = api.get_calendar()
    cal_ls = []
    for c in cal:
        cal_ls.append(c.date)
      
    cal_ls = pd.Series(cal_ls)
    return cal_ls

def get_data(api, stock_ls, start, end = None):
    
  # first_half
    indexes = np.arange(0,len(stock_ls),5)
    for i in range(len(indexes)-1):
        data = api.get_bars(
                      symbol =  stock_ls[indexes[i]:indexes[i+1]],
                      timeframe = TimeFrame.Day,
                      start = start,
                      end = end,
                      limit = 10000,
                      adjustment= 'raw',
                      ).df
        if i == 0:
            db = data
        else:
            db = db.append(data)
    return db

cal_ls = get_trading_cal(api)
tdy_date = dt.datetime.strftime(dt.datetime.today(), format = '%Y-%m-%d')
last_trading_date = cal_ls[cal_ls[cal_ls <= tdy_date].index[-2]].strftime('%Y-%m-%d')
df_data = get_data(api, op_df.Symbol.unique(), start = '2019-01-01', end = last_trading_date)

NameError: name 'op_df' is not defined

In [24]:
cal_ls[cal_ls[cal_ls <= tdy_date].index[-2]]

Timestamp('2022-05-26 00:00:00')

In [12]:
cal_ls

0       1970-01-02
1       1970-01-05
2       1970-01-06
3       1970-01-07
4       1970-01-08
           ...    
15131   2029-12-18
15132   2029-12-19
15133   2029-12-20
15134   2029-12-21
15135   2029-12-24
Length: 15136, dtype: datetime64[ns]

In [253]:
from alpaca_trade_api.rest import REST, TimeFrame
import os
import pandas as pd
from ta.momentum  import RSIIndicator  
from ta.trend import SMAIndicator
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import logging
import datetime as dt
import time 

from alpaca_trade_api.rest import REST, TimeFrame
import os
import pandas as pd
from ta.momentum  import RSIIndicator  
from ta.trend import SMAIndicator
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import logging
import datetime as dt
import time 

def RSI_SMA(data, symbol_list):
    alerts = []
    for symbol in symbol_list:
        df = data.loc[data.symbol == symbol] # get individual stock price


        # Calculate technical indicators
        rsi4 = RSIIndicator(df.close, window = 4)

        sma200 = SMAIndicator(df.close, 200)

        df['rsi4'] = rsi4.rsi()
        df['sma200'] = sma200.sma_indicator()
        
        try:
            latest = df.iloc[-1,:] # obtain latest indicators' values
        except:
            alerts.append([symbol,None, None])

        alerts.append([symbol,latest.rsi4, latest.sma200])

    alerts_df = pd.DataFrame(alerts, columns=['Symbol','RSI4','SMA200'])
    alerts_df.set_index('Symbol', inplace = True)

    return alerts_df

In [254]:
df_ta = RSI_SMA(df_data,op_df.Symbol.unique())

In [255]:
full_df = op_df.join(df_ta, on = 'Symbol')
full_df = full_df.set_index(np.arange(full_df.shape[0]))
full_df

,Symbol,Asset_class,Qty,Avg_entry_price,Change_today,Cost_basis,Current_price,Exchange,Lastday_price,Market_value,Side,Unrealized_intraday_pl ($),Unrealized_intraday_pl (%),Unrealized_pl ($),Unrealized_pl (%),RSI4,SMA200
0,DVY,us_equity,5,122.34,0.006890,611.68,130.06,NASDAQ,129.17,650.30,long,4.45,0.689014,38.62,6.313759,80.506461,122.21405
1,AMJ,us_equity,1,20.86,0.005535,20.86,21.80,ARCA,21.68,21.80,long,0.12,0.553506,0.94,4.506232,59.420934,19.41940
2,XLU,us_equity,8,72.06,0.007007,576.46,74.73,ARCA,74.21,597.84,long,4.16,0.700714,21.38,3.708844,84.603498,69.28570
3,VYM,us_equity,2,107.70,0.006422,215.39,111.26,ARCA,110.55,222.52,long,1.42,0.642243,7.13,3.310274,77.896669,109.43125
4,SCHD,us_equity,1,76.58,0.008215,76.58,78.55,ARCA,77.91,78.55,long,0.64,0.821461,1.97,2.572473,77.100409,77.77820
5,VTV,us_equity,1,142.90,0.007554,142.9,144.05,ARCA,142.97,144.05,long,1.08,0.755403,1.15,0.804759,77.600518,143.44320
6,VOE,us_equity,1,144.67,0.011059,144.67,145.36,ARCA,143.77,145.36,long,1.59,1.105933,0.69,0.476948,77.078914,145.89015
7,SDY,us_equity,2,126.72,0.007057,253.44,127.01,ARCA,126.12,254.02,long,1.78,0.705677,0.58,0.228851,75.915505,125.06975
8,SPYV,us_equity,1,40.59,0.008748,40.59,40.36,ARCA,40.01,40.36,long,0.35,0.874781,-0.23,-0.566642,78.227500,40.74630
9,IVE,us_equity,1,151.40,0.009324,151.4,150.46,ARCA,149.07,150.46,long,1.39,0.932448,-0.94,-0.620872,78.142396,151.93010


In [256]:
acc_details = api.get_account()
cash_balance = acc_details.cash
portfolio_value = acc_details.portfolio_value
acc_status = acc_details.status
long_market_value = acc_details.long_market_value
acc_details

Account({   'account_blocked': False,
    'account_number': 'PA3L5EBVA6LQ',
    'accrued_fees': '0',
    'buying_power': '15192.74',
    'cash': '5169.49',
    'created_at': '2022-02-22T15:40:56.226622Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '10023.25',
    'id': '089ead34-9c24-4a08-99fb-82a83d2403c8',
    'initial_margin': '2426.88',
    'last_equity': '9994.5',
    'last_maintenance_margin': '1447.5',
    'long_market_value': '4853.76',
    'maintenance_margin': '1456.128',
    'multiplier': '2',
    'non_marginable_buying_power': '5169.49',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '10023.25',
    'regt_buying_power': '15192.74',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '9965.64',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [257]:
float(portfolio_value) - float(long_market_value)

5169.49

In [258]:
full_df.Market_value.sum()

5182.92

In [259]:
full_df['cash_balance'] = cash_balance
full_df['portfolio_value'] = portfolio_value
full_df['long_market_value'] = long_market_value

In [260]:
client = pygsheets.authorize(service_file='algoquant-351309-ae021cd7392e.json')

sh = client.open_by_key('1hPxsuwdDvpZpQYG2yi_djwcDG1RDAw1RcS-QZ5prUPo')

wk2 = sh[1]
wk2.set_dataframe(full_df, 'A1')

In [261]:
wk3 = sh[3]
wk3.set_dataframe(df_agg, 'A1')